# Парсинг данных

В рамках данного проекта необходимо осуществить парсинг данных с веб-сайта. В качества веб-сайта взят habr.com. Необходимо осуществить парсинг лучших постов за все время. Данные для парсинга: 1) параметры - наименование статьи, дата загрузки, категория, к которой пост относится, и 2) целевой показатель - число просмотров. 

В дальнейшем можно рассмотреть не только лучшие за все ремя посты, но и увеличить выборку еще большим объемом повседневных постов.

На данный момент объем выборки должен составить 1000 статей (20 объектов на странице, всего 50 страниц).

<br>**План проекта**:
1. Импортирование библиотек
2. Парсинг данных
3. Сохранение базы данных

## 1. Импортирование библиотек

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
import time
import re
import pandas as pd

'''
from bs4 import BeautifulSoup
import os
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from itertools import cycle
'''

'\nfrom bs4 import BeautifulSoup\nimport os\nimport concurrent.futures\nfrom concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor\nfrom itertools import cycle\n'

## 2. Парсинг данных

In [38]:
firefox_opt = Options()
firefox_opt.page_load_strategy = 'eager'
driver = webdriver.Firefox(options=firefox_opt)

habr_url = 'https://habr.com/ru/top/alltime/'

In [39]:
# Функция для получения данных с одной веб-страницы
def get_page_data(page_num):
    # Заход на веб-страницу
    page_url = f'{habr_url}page{page_num}'
    driver.get(page_url)
    driver.implicitly_wait(5)
    
    # Прокрутка до конца страницы
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Наименования постов
    titles = [title.text for title in driver.find_elements(By.CLASS_NAME, 'tm-title__link')]
    
    # Количество просмотров
    view_counts_link = driver.find_elements(By.CLASS_NAME, 'tm-icon-counter__value')
    view_counts = []
    for view_count in view_counts_link:
        if 'M' in view_count.text:
            view_counts.append(float(re.split('M', view_count.text)[0]) * 1000)
        elif 'K' in view_count.text:
            view_counts.append(float(re.split('K', view_count.text)[0]))
    
    # Категории
    categories_link = driver.find_elements(By.CLASS_NAME, 'tm-article-snippet__hubs-container')
    categories = []
    for category in categories_link:
        categories.append([i for i in re.split('\n', category.text) if i != '*'])

    # Дата и время публикации
    dates_link = driver.find_elements(By.CSS_SELECTOR, 'time')
    dates = []
    for date in dates_link:
        if 'в' in date.text:
            dates.append(date.get_attribute('datetime'))
    
    df = pd.concat(
        [pd.Series(titles), pd.Series(view_counts), pd.Series(categories), pd.Series(dates)],
        axis=1
    )
    
    df.columns = ['title', 'view_count', 'category', 'date']
    
    return df

In [40]:
df_posts = pd.DataFrame(columns=['title', 'view_count', 'category', 'date'])

for n in range(1, 51):
    df_posts = df_posts.append(get_page_data(n), ignore_index=True)

driver.quit()

25
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20


In [50]:
df_posts = df_posts[~df_posts['title'].isna()].reset_index(drop=True)

In [53]:
df_posts[df_posts['view_count'].isna()]

,title,view_count,category,date
339,Заблуждения программистов о картах,NaN,"[Глобальные системы позиционирования, Программ...",2022-09-01T14:41:54.000Z
739,Пара лайфхаков нашего интернет-магазина,NaN,[Блог компании Мосигра],2013-11-05T04:57:08.000Z
899,"Весенний апдейт, ч.1 «Торт»",NaN,"[Блог компании Хабр, Habr]",2011-03-17T16:48:28.000Z


## 3. Сохранение данных

In [ ]:
df_posts.to_csv('posts_data.csv')